In [ ]:
import requests
import pymongo
from pymongo import MongoClient


mongodb_server = 'mongodb://{change it, Your mongodb address}'
mongodb_port = {change it, mongodb port}
mongo_client = MongoClient(mongodb_server, mongodb_port, username='{mongodb username}', password='{mongodb password}')

tmdb = mongo_client['tmdb']

collection_names = tmdb.list_collection_names()


new_mongo_client = MongoClient(mongodb_server, mongodb_port, username='root', password='root')
new_db = new_mongo_client['tmdblast']

new_collection = new_db['trailer']
new_d_collection = new_db['detail']
new_p_collection = new_db['provider']
new_c_collection = new_db['cast']

headers = {
    "accept": "application/json",
    "Authorization": "{change it, your tmdb api}"
}

for collection_name in collection_names:
    collection = tmdb[collection_name]
    for document in collection.find():
        i = document['id']
        detail_url = f"https://api.themoviedb.org/3/movie/{i}?append_to_response=%2C&language=ko-KR"
        trailer_url = f"https://api.themoviedb.org/3/movie/{i}/videos?language=ko-KR"
        provider_url = f"https://api.themoviedb.org/3/movie/{i}/watch/providers"
        actor_url = f"https://api.themoviedb.org/3/movie/{i}/credits?language=ko-KR"
        try:
            detail_response = requests.get(detail_url, headers=headers)
            trailer_response = requests.get(trailer_url, headers=headers)
            provider_response = requests.get(provider_url, headers=headers)
            actor_response = requests.get(actor_url, headers=headers)

            if detail_response.status_code == 404:
                continue

            detail_data = detail_response.json()
            trailer_data = trailer_response.json()
            provider_data = provider_response.json()
            actor_data = actor_response.json()

            caster_data = actor_data.get("cast", [])
            crew_data = actor_data.get("crew", [])

            if "status_code" in detail_data:
                continue

            movie_ids = [cast['id'] for cast in actor_data['cast']]
            movie_ids += [crew['id'] for crew in actor_data['crew']]

            caster_documents = [
                {
                    "id": cast.get("id"),
                    "gender": cast.get("gender"),
                    "known_for_department": cast.get("known_for_department"),
                    "name": cast.get("name"),
                    "popularity": cast.get("popularity"),
                    "profile_path": cast.get("profile_path"),
                    "cast_id": cast.get("cast_id"),
                    "character": cast.get("character")
                }
                for cast in caster_data
            ]

            crew_documents = [
                {
                    "id": crew.get("id"),
                    "gender": crew.get("gender"),
                    "known_for_department": crew.get("known_for_department"),
                    "name": crew.get("name"),
                    "popularity": crew.get("popularity"),
                    "profile_path": crew.get("profile_path"),
                    "credit_id": crew.get("credit_id"),
                    "department": crew.get("department"),
                    "job": crew.get("job")
                }
                for crew in crew_data
            ]

            # Insert data into the new collections
            new_collection.insert_one(trailer_data)
            new_d_collection.insert_one(detail_data)
            new_p_collection.insert_one(provider_data)
            new_c_collection.insert_one({"id": i, "caster": caster_documents, "crew": crew_documents})

            print(detail_data['title'], detail_data['id'])

        except requests.exceptions.RequestException as e:
            print("ERROR to api:", str(e))
            continue

        except json.JSONDecodeError as e:
            print("ERROR JSON decoding:", str(e))
            continue

        except Exception as e:
            print("ERROR else:", str(e))
            continue